In [1]:
import pandas as pd
import numpy as np
import pickle
root="data/"

In [2]:
df = {}
for k in ["train", "val", "test"]:
    df[k] = {}
    for c in ["bong"]:
        df[k][c] = pd.read_excel(root+f"{c}_{k}.xlsx", index_col=0)

In [3]:
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier

In [4]:
cols_bong = [c for c in df["train"]["bong"].columns if c not in ["id", "label"]]

X_train = df["train"]["bong"][cols_bong]
X_val = df["val"]["bong"][cols_bong]
X_test = df["test"]["bong"][cols_bong]

X_train = X_train.loc[:, ~X_train.columns.duplicated()]
X_val = X_val.loc[:, ~X_val.columns.duplicated()]
X_test = X_test.loc[:, ~X_test.columns.duplicated()]

y_train = (df["train"]["bong"]["label"]).apply(int)
y_val = (df["val"]["bong"]["label"]).apply(int)
y_test = (df["test"]["bong"]["label"]).apply(int)

In [5]:
train_test_sample = pd.concat([
    pd.concat([X_train, y_train], axis=1), 
    pd.concat([X_val, y_val], axis=1)
    ], ignore_index=True)
train_test_sample

,00,000,000 35,000 700,000 pesos,02,10,10 10,10 de,10 es,...,vuelto,ya,ya que,yo,yo creo,yo lo,yo multiplique,yo reste,yo sume,label
0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
3,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.581128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14452,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
14453,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.251441,0.0,0.0,0.0,0.0,0.0,0
14454,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0
14455,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0


In [6]:
import json

In [7]:
F_features = np.load("output/bong_features.npy")

In [8]:
features, counts = np.unique(F_features, return_counts=True)
k = 5
new_best_features = features[counts>=k]

X_train_val = train_test_sample.drop(columns="label")
y_train_val = train_test_sample["label"]

clf = DecisionTreeClassifier(random_state=42, max_depth=5)
clf.fit(X_train_val[new_best_features], y_train_val)
print(clf.score(X_train_val[new_best_features], y_train_val))
print(classification_report(y_test, clf.predict(X_test[new_best_features]), digits=4))
o = classification_report(y_test, clf.predict(X_test[new_best_features]), digits=4, output_dict=True)
o

0.8725185031472643
              precision    recall  f1-score   support

           0     0.8123    1.0000    0.8964       541
           1     1.0000    0.0809    0.1497       136

    accuracy                         0.8154       677
   macro avg     0.9062    0.5404    0.5230       677
weighted avg     0.8500    0.8154    0.7464       677



{'0': {'precision': 0.8123123123123123,
  'recall': 1.0,
  'f1-score': 0.8964374482187241,
  'support': 541},
 '1': {'precision': 1.0,
  'recall': 0.08088235294117647,
  'f1-score': 0.14965986394557826,
  'support': 136},
 'accuracy': 0.8153618906942393,
 'macro avg': {'precision': 0.9061561561561562,
  'recall': 0.5404411764705882,
  'f1-score': 0.5230486560821511,
  'support': 677},
 'weighted avg': {'precision': 0.850016190488864,
  'recall': 0.8153618906942393,
  'f1-score': 0.7464200900781808,
  'support': 677}}